# House

In [1]:
using Random
using LinearAlgebra
rng = MersenneTwister(2018)
import LinearAlgebra.dot
import LinearAlgebra.norm
import LinearAlgebra.cond
import LinearAlgebra.triu
import LinearAlgebra.qr
import LinearAlgebra.UniformScaling;

Random.seed!(69)

e = 1e-10 * randn()
A = [1 1 1; e 0 0; 0 e 0; 0 0 e]
A = float(A)
A0 = copy(A)

4×3 Matrix{Float64}:
 1.0          1.0          1.0
 8.44369e-11  0.0          0.0
 0.0          8.44369e-11  0.0
 0.0          0.0          8.44369e-11

In [2]:
function house(x)
    """Computes the Householder transformation for input vector x"""
    sigma = dot(x[2:end],x[2:end])
    v = copy(x)

    if sigma == 0
        beta = 0
        return beta, v
    end

    sq = sqrt(x[1]^2 + sigma)
    if x[1] > 0
        v[1] += sq
    else
        v[1] -= sq
    end

    beta = 2.0 / (v[1]^2 + sigma)

    return beta, v
end


house (generic function with 1 method)

In [3]:
m = size(A,1)
n = size(A,2)
vA = zeros(n)
E = Matrix{Float64}(I, m, m)
kend = (m > n ? n : m-1)
Q = copy(E)
for k=1:kend
    beta, v = house(A[k:end,k])
G = Matrix{Float64}(I, m-k+1, m-k+1) - beta*v*v'

P = copy(E)
P = float(P)
P[k:end, k:end] = G

Q = Q*P

    for j=k:n
        vA[j] = 0.0
        for i=k:m
            vA[j] += v[i-k+1] * A[i,j]
        end
        vA[j] *= beta
    end
    for j=k:n, i=k:m
        A[i,j] -= v[i-k+1] * vA[j]
    end

v = v/v[1]
    A[k+1:end,k] = v[2:end]
end

In [4]:
R = triu(A)

4×3 Matrix{Float64}:
 -1.0  -1.0          -1.0
  0.0   1.19412e-10   5.97059e-11
  0.0   0.0           1.03414e-10
  0.0   0.0           0.0

In [5]:
display("Reconstruction error after using house")
display(norm(Q*R - A0))

display("Check whether Q is orhonormal")
display(norm(Q' * Q - Matrix(I, 4, 4)))
;

"Reconstruction error after using house"

1.9236393923411068e-26

"Check whether Q is orhonormal"

3.499222384778056e-16

# CGS

In [6]:
using Random
using LinearAlgebra
rng = MersenneTwister(2018)
import LinearAlgebra.dot
import LinearAlgebra.norm
import LinearAlgebra.triu
import LinearAlgebra.diag
;
Random.seed!(69)

e = 1e-10 * randn()
A = [1 1 1; e 0 0; 0 e 0; 0 0 e]
A = float(A)
A0 = copy(A)
B0 = copy(A)
C0 = copy(A)

4×3 Matrix{Float64}:
 1.0          1.0          1.0
 8.44369e-11  0.0          0.0
 0.0          8.44369e-11  0.0
 0.0          0.0          8.44369e-11

In [7]:
m = size(A,1)
n = size(A,2)
@assert m >= n
R = zeros(Float64, n,n)
for j=1:n
    for i=1:j-1, k=1:m
        R[i,j] += A[k,i] * A[k,j]
    end
    for i=1:j-1, k=1:m
        A[k,j] -= A[k,i] * R[i,j]
    end
    R[j,j] = norm( A[:,j] )
    A[:,j] /= R[j,j]
end

In [8]:
display("Reconstruction error after using house")
display(norm(A*R - A0))

display("Check whether Q is orhonormal")
display(norm(A' * A - Matrix(I, 3, 3)))
;

"Reconstruction error after using house"

5.520603538388541e-27

"Check whether Q is orhonormal"

0.7071067811865477